<a href="https://colab.research.google.com/github/Arijit-datascience/pytorch_cifar10/blob/main/pytorch_cifar10_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.optim as optim

from torchsummary import summary
from torch.optim.lr_scheduler import StepLR, OneCycleLR, ReduceLROnPlateau

import os
import numpy as np

In [2]:
%%capture
!git clone https://github.com/Arijit-datascience/pytorch_cifar10
!pip install albumentations==1.0.0

In [3]:
from pytorch_cifar10 import main as main
from pytorch_cifar10.model import resnet as models
from pytorch_cifar10.utils import utils as utils

In [4]:
norm_mean,norm_std = utils.cifar10_mean_std()
train_transform, test_transform = utils.get_transforms(norm_mean,norm_std)

# Extract and transform the data
train_set,test_set = utils.get_datasets(train_transform,test_transform)

# Targets -> ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
classes = train_set.classes

# Dataloader Arguments & Test/Train Dataloaders - Load part of ETL
train_loader,test_loader = utils.get_dataloaders(train_set,test_set)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
(0.492, 0.483, 0.447) (0.247, 0.243, 0.262)
Files already downloaded and verified
Files already downloaded and verified
CUDA Available? True


In [ ]:
# Explore sample images
utils.show_sample_images(train_loader, classes, .5, .25, num_of_images = 10)

In [5]:
# Model summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = models.ResNet18("LN").to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
         GroupNorm-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
         GroupNorm-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
         GroupNorm-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
         GroupNorm-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
        GroupNorm-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 32, 32]          73,728
        GroupNorm-14          [-1,

In [ ]:
if __name__=='__main__':
    
    EPOCH = 40
    batch_size = 256
    l1_factor = 0
    l2_factor = 0.0001
    lr_value = 0.001
    mt_val = 0.9
    
    cuda = torch.cuda.is_available()
    device = torch.device("cuda" if cuda else "cpu")
    
    utils.seed_everything(1)
    
    optimizer = optim.Adam(model.parameters(), lr=lr_value, weight_decay=l2_factor)
    scheduler = ReduceLROnPlateau(optimizer, factor=0.2, patience=3,verbose=True,mode='max')
    
    print("------------------------------------------")
    train_loss, test_loss, train_acc, test_acc, test_pred, target_pred, target_data = main.main(EPOCH, model, device, train_loader, test_loader, optimizer, scheduler, l1_factor)

------------------------------------------

Epoch 1 : 
Train set: Average loss: 1.7728, Accuracy: 21.10

Test set: Average loss: 0.014, Accuracy: 33.73

Epoch 2 : 
Train set: Average loss: 1.4157, Accuracy: 35.34

Test set: Average loss: 0.012, Accuracy: 43.81

Epoch 3 : 
Train set: Average loss: 1.5322, Accuracy: 42.16

Test set: Average loss: 0.011, Accuracy: 49.58

Epoch 4 : 
Train set: Average loss: 1.4792, Accuracy: 47.02

Test set: Average loss: 0.010, Accuracy: 51.22

Epoch 5 : 
Train set: Average loss: 1.4915, Accuracy: 51.08

Test set: Average loss: 0.009, Accuracy: 57.00
Epoch     5: reducing learning rate of group 0 to 2.0000e-04.

Epoch 6 : 
Train set: Average loss: 0.9251, Accuracy: 58.44

Test set: Average loss: 0.008, Accuracy: 64.06

Epoch 7 : 
Train set: Average loss: 1.1173, Accuracy: 60.51

Test set: Average loss: 0.008, Accuracy: 65.16

Epoch 8 : 
Train set: Average loss: 0.9210, Accuracy: 62.30

Test set: Average loss: 0.007, Accuracy: 68.15

Epoch 9 : 


In [ ]:
wrong_predictions = utils.wrong_predictions(model,test_loader)
plot_misclassified(wrong_predictions)